In [1]:
%matplotlib inline
import numpy as np
import sklearn.decomposition
import sklearn.feature_extraction
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import bottleneck as bn
import itertools

###Fake Testing

In [2]:
# arbitrary 6x3 matrix (constructed from an example at http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html with a random column concatenated)
M = np.concatenate((np.array([[1,1], [2, 1], [3, 1.2], [4, 1], [5, 0.8], [6, 1]]),map(lambda x : [x], np.round(np.random.random_sample(6),1)+1)),1)
M
H = sklearn.decomposition.NMF(n_components=2).fit(M).components_
W = sklearn.decomposition.NMF(n_components=2).fit_transform(M)
print H
print
print W

[[  3.08686011e+00   5.19450651e-01   6.66252184e-01]
 [  1.70115150e-06   7.69621201e-01   1.34443684e+00]]

[[ 0.32250324  1.21106046]
 [ 0.64847343  0.81094823]
 [ 0.97662223  0.47380269]
 [ 1.29791641  0.23617409]
 [ 1.61546449  0.33672095]
 [ 1.94355571 -0.        ]]


In [3]:
# a 20x4 example in which there are explicit types (that are unknown to the algorithm)
n = 20
types = np.trunc(np.random.random_sample(n)*3)
means = [[1,2,3,4],[1,5,4,1],[10,1,2,8]]
stddevs = 0.5 * np.ones([3,4])
def gen(t):
    mean = means[t]
    sd = stddevs[t]
    sz = len(mean)
    return list(np.random.randn(sz) * sd + mean)
# generate a data matrix
M = np.round(map(gen, map(int, types)),3)

M_tfidf = sklearn.feature_extraction.text.TfidfTransformer().fit_transform(M).toarray()
print M
print
print M_tfidf
print
print

# run NMF
fit = sklearn.decomposition.NMF(n_components=3).fit(M)
H = fit.components_
W = fit.transform(M)
#print H
results = list(np.concatenate((10*W,map(lambda x:[x],types)), 1))
results.sort(key = lambda row : row[3])
results = np.array(map(lambda row : list(np.round(row,3)), results))
print results
#print map(lambda row : np.argmax(np.round(list(row[0:3]),3)), results)
# predicted clusters
print map(lambda row : np.argmax(np.round(list(row[0:3]),3)), W)
# true clusters
print map(int, types)
# use fit from above to classify a new point
print fit.transform([[1,2,4,5]])

print
print

fit = sklearn.decomposition.NMF(n_components=3).fit(M_tfidf)
H = fit.components_
W = fit.transform(M_tfidf)
#print H
results = list(np.concatenate((10*W,map(lambda x:[x],types)), 1))
results.sort(key = lambda row : row[3])
results = np.array(map(lambda row : list(np.round(row,3)), results))
print results
#print map(lambda row : np.argmax(np.round(list(row[0:3]),3)), results)
# predicted clusters
print map(lambda row : np.argmax(np.round(list(row[0:3]),3)), W)
# true clusters
print map(int, types)

[[  0.912   4.764   4.271   1.035]
 [  1.16    2.034   3.073   3.923]
 [  0.688   5.084   3.982   1.942]
 [  0.577   2.52    3.041   3.083]
 [  1.491   5.25    3.009  -0.091]
 [  1.322   4.423   3.577   0.976]
 [  1.546   2.266   2.607   3.338]
 [ 10.782   0.992   1.418   8.514]
 [  0.775   5.259   4.441  -0.258]
 [  9.641   1.76    1.764   8.594]
 [  1.919   5.395   2.8     0.884]
 [  0.681   1.33    2.459   3.3  ]
 [  0.538   5.633   4.439   1.07 ]
 [  9.833   0.901   1.539   7.052]
 [  1.205   4.392   3.42    0.557]
 [  9.67    0.9     1.321   7.563]
 [  1.188   2.299   3.261   3.432]
 [  0.425   5.228   3.779   0.846]
 [ 10.399   0.473   1.429   7.663]
 [  9.453   0.446   3.176   8.562]]

[[ 0.139338    0.72785771  0.65253574  0.1581303 ]
 [ 0.21067934  0.36941532  0.55811862  0.71249572]
 [ 0.10149735  0.75001823  0.58744544  0.28649398]
 [ 0.11440691  0.49966277  0.60296607  0.61129378]
 [ 0.23921788  0.84231646  0.48276766 -0.01460015]
 [ 0.22327349  0.74700351  0.60412199  0.16

ValueError: Negative values in data passed to NMF.fit

###Fitting Model with Training Data

In [4]:
%%time
# read in data from csv
noun_train_mat = np.loadtxt("noun_train_mat.csv", delimiter = ",")
# use TF-IDF to scale each document's vector to have norm 1 and place a lower weight on very common words
tf_idf_fit = sklearn.feature_extraction.text.TfidfTransformer().fit(noun_train_mat)
noun_train_mat = tf_idf_fit.transform(noun_train_mat).toarray()

Wall time: 32.3 s


In [45]:
%%time
# compute NMF fit
NMF_fit = sklearn.decomposition.NMF(n_components=14, init='nndsvda').fit(noun_train_mat)
H = NMF_fit.components_
W = NMF_fit.transform(noun_train_mat)
# contains a tuple (i,j) if document i is in cluster j, for each document
clusters = map(np.argmax, W)
# list of the documents in each cluster
cluster_lists = [[i for i,j in enumerate(clusters) if j==cluster] for cluster in range(14)]

Wall time: 1min 7s


In [46]:
# classify each document into the category that fits it best
print 'Number of documents per category:', [sum([x==i for x in clusters]) for i in range(14)]

Number of documents per category: [100, 185, 74, 59, 68, 145, 44, 68, 88, 53, 35, 13, 26, 152]


###Key Words for Each Cluster

In [47]:
# load vocab from csv
noun_vocab = np.loadtxt("noun_vocab.csv", delimiter=",", dtype="str")
noun_vocab = [(int(i),j) for i,j in noun_vocab]
id2noun = dict(noun_vocab)

In [48]:
# find and output the 5 most important words for each category
num_best = 5
best_indices = map(lambda v : list(bn.argpartsort(-v,num_best)[0:num_best]), H)
best_words = [[id2noun[i] for i in lst] for lst in best_indices]
best_words

[['court', 'district', 'petitioner', 'appeal', 'habea'],
 ['act', 'respondent', 'action', 'court', 'title'],
 ['labor', 'union', 'board', 'employee', 'employer'],
 ['warrant', 'police', 'search', 'officer', 'petitioner'],
 ['property', 'tax', 'revenue', 'income', 'busines'],
 ['jury', 'sentence', 'defendant', 'trial', 'offense'],
 ['carrier', 'railroad', 'icc', 'rate', 'commerce'],
 ['attorney', 'alien', 'general', 'brief', 'cause'],
 ['commission', 'price', 'act', 'sale', 'company'],
 ['contract', 'arbitration', 'agreement', 'government', 'contractor'],
 ['plan', 'board', 'school', 'student', 'education'],
 ['patent', 'invention', 'art', 'claim', 'royalty'],
 ['master', 'decree', 'orig ', 'entry', 'boundary'],
 ['state', 'court', 'statute', 'law', 'amendment']]

In [49]:
# can probably ignore this stuff -- it's a different way to figure out the best words in each category, but it's not working
# and produces nonsense
width = len(M[0])
best_words = np.zeros(14)
for i in range(14):
    # arbitrary high initial value
    w = np.ones(width) * 5000
    #w_val = np.ones(width)
    for j in range(width):
        for k in range(14):
            if i <> k and H[k][j] > 0:
                if w[j] >= float(H[i][j])/H[k][j]:
                    w[j] = float(H[i][j])/H[k][j]
    best_words[i] = np.argmax(w)
print map(lambda x : id2noun[int(x)], best_words)

['narcotic', 'fdca', 'middleman', 'narcotic', 'city the', 'middleman', 'city the', 'middleman', 'fdca', 'narcotic', 'city the', 'fdca', 'city the', 'city the']


###Comparing with Supreme Court Database's Topic Areas

In [50]:
# read in SC Database's issue areas from csv
noun_train_issue_areas = np.loadtxt("noun_train_issue_areas.csv", delimiter = ",", dtype="int")
# zero-index the array
noun_train_issue_areas = noun_train_issue_areas - 1
print 'Number of documents per category:', [sum([x==i for x in noun_train_issue_areas]) for i in range(14)]

# convert the nx1 issue areas vector into a nx14 matrix of dummies, in case that's ever useful
noun_train_issue_areas_dummy = np.array(map(lambda area : np.eye(1,14,area)[0], noun_train_issue_areas))

Number of documents per category: [238, 188, 96, 36, 10, 8, 64, 242, 131, 56, 10, 30, 1, 0]


In [52]:
# create a 14x14 matrix (where each row is an NMF cluster and each column is an SCDB cluster) measuring the degree of
# related-ness between each cluster pair
compare_mat = map(lambda r : map(int, r), np.zeros((14,14)))
# first, assign the (i,j) element in the matrix to the number of cases in NMF cluster i and SCDB cluster j, for each (i,j)
for i,j in zip(clusters, noun_train_issue_areas):
    compare_mat[i][j] += 1
# normalize each row to have a sum of 1
compare_mat = map(lambda row : map(float,row) / sum(row), np.array(compare_mat))

### commented out because I realized this method of assignment does the same thing as the one-line assignment below
#assignments = -1 * np.ones(14)
#compare_mat_flat = [x for lst in compare_mat for x in lst]
#while min(assignments) == -1:
#    max_ind = np.argmax(compare_mat_flat)
#    row = max_ind / 14
#    if assignments[row] == -1:
#        column = max_ind - 14 * row
#        assignments[row] = column
#    else:
#        compare_mat_flat[max_ind] = -1
#assignments = map(int, assignments)
#print assignments

# assign each NMF cluster to a (not necessarily unique) SCDB cluster by picking the highest element of each row of compare_mat
assignments = map(np.argmax, compare_mat)
print assignments
# this would be the equivalent way of assigning each SCDB cluster to an NMF cluster; this is printed here but not used anywhere
print map(np.argmax, np.array(compare_mat).T)

[8, 7, 6, 0, 7, 0, 7, 1, 7, 7, 1, 7, 10, 2]
[3, 10, 13, 13, 13, 7, 2, 11, 0, 12, 12, 4, 4, 0]


In [51]:
### NOTE: probably don't run this -- it generates assignments that have very low accuracy
# use Hungarian algorithm (which computes assignments that minimize cost) to assign each NMF cluster to an SCDB cluster (without
# any collisions)
import munkres
m = munkres.Munkres()
assignments = dict(m.compute(-np.array(compare_mat)))
print assignments

ValueError: operands could not be broadcast together with shapes (14,) (16,) (14,) 

In [53]:
# use the assignments above to convert the clusters into equivalent SCDB categories for each document
new_clusters = map(lambda cluster : assignments[cluster], clusters)
# compute and output accuracy rate
correct = map(lambda (c1,c2) : c1==c2, zip(new_clusters, noun_train_issue_areas))
print float(sum(correct)) / len(correct)

0.490990990991


###Use Rand Index to Compute Accuracy on Training Data

Each pair of documents is called a true positive ($tp$) if the two documents end up in the same cluster correctly, a true negative ($tn$) if the documents end up in different clusters correctly, a false positive ($fp$) if the documents are placed in the same cluster but should have been in different clusters, and a false negative ($fn$) if the documents are placed in different clusters but should have been in the same cluster.

The Rand index (measuring the similarity between the NMF clustering and the SCDB categories) is defined as $\frac{tp+tn}{tp+tn+fp+fn}$.

In [54]:
%%time
if 'cluster_pairs_model' in globals():
    del cluster_pairs_model
if 'cluster_pairs_actual' in globals():
    del cluster_pairs_actual

# use itertools to create pairs of documents, using the NMF clusters and SCDB categories
cluster_pairs_model = itertools.product(clusters, clusters)
cluster_pairs_actual = itertools.product(noun_train_issue_areas, noun_train_issue_areas)
# loop through all document pairs to compute tp, tn, fp, and fn
(tp,tn,fp,fn) = 0, 0, 0, 0
for pair1,pair2 in zip(cluster_pairs_model, cluster_pairs_actual):
    if (pair1[0]==pair1[1]):
        if (pair2[0]==pair2[1]):
            tp += 1
        else:
            fp += 1
    else:
        if (pair2[0]==pair2[1]):
            fn += 1
        else:
            tn += 1

# output tp, tn, fp, and fn
print tp,tn,fp,fn
# compute and output Rand index using formula from above
rand_index_train = float(tp+tn)/(tp+tn+fp+fn)
print rand_index_train

39504 963864 81614 147118
0.814355977599
Wall time: 1.69 s


###Applying Model to Test Data

In [15]:
%%time
# read in test data from csv
noun_test_mat = np.loadtxt("noun_test_mat.csv", delimiter = ",")
# use TF-IDF fit from training data to scale each document's vector to have norm 1 and place a lower weight on very common words
noun_test_mat = tf_idf_fit.transform(noun_test_mat).toarray()

Wall time: 12.2 s


In [16]:
%%time
# use NMF fit from training data to cluster test observations
W_test = NMF_fit.transform(noun_test_mat)
clusters_test = map(np.argmax, W_test)
cluster_lists_test = [[i for i,j in enumerate(clusters_test) if j==cluster] for cluster in range(14)]

Wall time: 3.13 s


###Compare Test Data Results to SC Database Categories

In [17]:
# read in SC Database's issue areas from csv
noun_test_issue_areas = np.loadtxt("noun_test_issue_areas.csv", delimiter = ",", dtype="int")
# zero-index the array
noun_test_issue_areas = noun_test_issue_areas - 1
print 'Number of documents per category:', [sum([x==i for x in noun_test_issue_areas]) for i in range(14)]

Number of documents per category: [104, 95, 26, 22, 3, 7, 18, 96, 63, 25, 5, 17, 4, 0]


In [18]:
# use the assignments above to convert the clusters into equivalent SCDB categories for each document
new_clusters_test = map(lambda cluster : assignments[cluster], clusters_test)
# compute and output accuracy rate
correct_test = map(lambda (c1,c2) : c1==c2, zip(new_clusters_test, noun_test_issue_areas))
print float(sum(correct_test)) / len(correct_test)

0.455670103093


###Use Rand Index to Compute Accuracy on Test Data

In [19]:
%%time
if 'cluster_test_pairs_model' in globals():
    del cluster_test_pairs_model
if 'cluster_test_pairs_actual' in globals():
    del cluster_test_pairs_actual

# use itertools to create pairs of documents, using the NMF clusters and SCDB categories
cluster_test_pairs_model = itertools.product(clusters_test, clusters_test)
cluster_test_pairs_actual = itertools.product(noun_test_issue_areas, noun_test_issue_areas)
# loop through all document pairs to compute tp, tn, fp, and fn
(tp,tn,fp,fn) = 0, 0, 0, 0
for pair1,pair2 in zip(cluster_test_pairs_model, cluster_test_pairs_actual):
    if (pair1[0]==pair1[1]):
        if (pair2[0]==pair2[1]):
            tp += 1
        else:
            fp += 1
    else:
        if (pair2[0]==pair2[1]):
            fn += 1
        else:
            tn += 1

# output tp, tn, fp, and fn
print tp,tn,fp,fn
# compute and output Rand index using formula from above
rand_index_test = float(tp+tn)/(tp+tn+fp+fn)
print rand_index_test

7869 184414 15288 27654
0.817442873844
Wall time: 344 ms


###Compute Optimal Number of Clusters on Training Data (based on SCDB)
Note: In reality, we should do this using one or more validation sets; I've just done this for the time being.

In [76]:
%%time
num_cluster_lst = [5, 10, 14, 15, 20, 25, 30]

results = dict()
for num_clusters in num_cluster_lst:
    NMF_fit = sklearn.decomposition.NMF(n_components=num_clusters, init='nndsvda').fit(noun_train_mat)
    H = NMF_fit.components_
    W = NMF_fit.transform(noun_train_mat)
    # contains a tuple (i,j) if document i is in cluster j, for each document
    clusters = map(np.argmax, W)
    # list of the documents in each cluster
    cluster_lists = [[i for i,j in enumerate(clusters) if j==cluster] for cluster in range(num_clusters)]
    
    # find the 10 most important words for each category
    num_best = 10
    best_indices = map(lambda v : list(bn.argpartsort(-v,num_best)[0:num_best]), H)
    best_words = [[id2noun[i] for i in lst] for lst in best_indices]
    
    # create a k x 14 matrix (where each row is an NMF cluster and each column is an SCDB cluster) measuring the degree of
    # related-ness between each cluster pair
    compare_mat = map(lambda r : map(int, r), np.zeros((num_clusters,14)))
    # first, assign the (i,j) element in the matrix to the number of cases in NMF cluster i and SCDB cluster j, for each (i,j)
    for i,j in zip(clusters, noun_train_issue_areas):
        compare_mat[i][j] += 1
    # normalize each row to have a sum of 1
    compare_mat = map(lambda row : map(float,row) / sum(row), np.array(compare_mat))
    # assign each NMF cluster to a (not necessarily unique) SCDB cluster by picking the highest element of each row of compare_mat
    assignments = map(np.argmax, compare_mat)
    # use the assignments above to convert the clusters into equivalent SCDB categories for each document
    new_clusters = map(lambda cluster : assignments[cluster], clusters)
    # compute and output accuracy rate
    correct = map(lambda (c1,c2) : c1==c2, zip(new_clusters, noun_train_issue_areas))
    accuracy = float(sum(correct)) / len(correct)

    if 'cluster_pairs_model' in globals():
        del cluster_pairs_model
    if 'cluster_pairs_actual' in globals():
        del cluster_pairs_actual
    # use itertools to create pairs of documents, using the NMF clusters and SCDB categories
    cluster_pairs_model = itertools.product(clusters, clusters)
    cluster_pairs_actual = itertools.product(noun_train_issue_areas, noun_train_issue_areas)
    # loop through all document pairs to compute tp, tn, fp, and fn
    (tp,tn,fp,fn) = 0, 0, 0, 0
    for pair1,pair2 in zip(cluster_pairs_model, cluster_pairs_actual):
        if (pair1[0]==pair1[1]):
            if (pair2[0]==pair2[1]):
                tp += 1
            else:
                fp += 1
        else:
            if (pair2[0]==pair2[1]):
                fn += 1
            else:
                tn += 1
    # compute Rand index using formula from above
    rand_index = float(tp+tn)/(tp+tn+fp+fn)
    
    results[num_clusters] = (NMF_fit, clusters, cluster_lists, best_words, new_clusters, accuracy, (tp,tn,fp,fn), rand_index)


Wall time: 31.5 s
